In [1]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import plot_histogram, plot_state_city
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Sampler

n=3

d:\Development\Python\3.7.5\lib\site-packages\pkg_resources\__init__.py:126: PkgResourcesDeprecationWarning: p is an invalid version and will not be supported in a future release
  PkgResourcesDeprecationWarning,


In [2]:
service = QiskitRuntimeService()
options = Options(optimization_level=1)
options.execution.shots = 1000

In [3]:
def cnx(qc, *qubits):
    if len(qubits) >= 3:
        last = qubits[-1]
        # A matrix: (made up of a  and Y rotation, lemma4.3)
        qc.crz(np.pi/2, qubits[-2], qubits[-1])
        qc.u(np.pi/2, 0, 0,qubits[-1])
        
        # Control not gate
        cnx(qc,*qubits[:-2],qubits[-1])
        
        # B matrix (pposite angle)
        qc.u(-np.pi/2, 0, 0, qubits[-1])
        
        # Control
        cnx(qc,*qubits[:-2],qubits[-1])
        
        # C matrix (final rotation)
        qc.crz(-np.pi/2,qubits[-2],qubits[-1])
    elif len(qubits)==3:
        qc.ccx(*qubits)
    elif len(qubits)==2:
        qc.cx(*qubits)

def increment_gate(qwc, q, subnode):
    cnx(qwc, subnode[0], q[2], q[1], q[0])
    cnx(qwc, subnode[0], q[2], q[1])
    cnx(qwc, subnode[0], q[2])
    qwc.barrier()
    return qwc

def decrement_gate(qwc, q, subnode):
    qwc.x(subnode[0])
    qwc.x(q[2])
    qwc.x(q[1])
    cnx(qwc, subnode[0], q[2], q[1], q[0])
    qwc.x(q[1])
    cnx(qwc, subnode[0], q[2], q[1])
    qwc.x(q[2])
    cnx(qwc, subnode[0], q[2])
    qwc.x(subnode[0])
    return qwc

def ibmsim(circ):
    with Session(service=service, backend="ibmq_qasm_simulator") as session:
        sampler = Sampler(session=session, options=options)
        job = sampler.run(circuits=circ)
        print(f"Job ID is {job.job_id()}")
        result = job.result()
        print("Job Done!")
        return result

In [4]:
qnodes = QuantumRegister(n,'qc')
qsubnodes = QuantumRegister(1,'qanc')
csubnodes = ClassicalRegister(1,'canc')
cnodes = ClassicalRegister(n,'cr')

qwc = QuantumCircuit(qnodes, qsubnodes, cnodes, csubnodes)

def runQWC(qwc, times):
    for i in range(times):
        qwc.h(qsubnodes[0])
        increment_gate(qwc, qnodes, qsubnodes)
        decrement_gate(qwc, qnodes, qsubnodes)
        qwc.measure(qnodes, cnodes)

    return qwc

step = 1
qwc = runQWC(qwc, step)
print(qwc.draw(output="text"))
#result = ibmsim(qwc)

#print(result.get_counts(circ))

        ┌─────────┐┌────────────┐┌─────────┐┌────────────┐┌───┐┌─────────────┐»
  qc_0: ┤ Rz(π/2) ├┤ U(π/2,0,0) ├┤ Rz(π/2) ├┤ U(π/2,0,0) ├┤ X ├┤ U(-π/2,0,0) ├»
        └────┬────┘└────────────┘└────┬────┘└────────────┘└─┬─┘└─────────────┘»
  qc_1: ─────■────────────────────────┼─────────────────────┼─────────────────»
                                      │                     │                 »
  qc_2: ──────────────────────────────■─────────────────────┼─────────────────»
           ┌───┐                                            │                 »
  qanc: ───┤ H ├────────────────────────────────────────────■─────────────────»
           └───┘                                                              »
  cr: 3/══════════════════════════════════════════════════════════════════════»
                                                                              »
canc: 1/══════════════════════════════════════════════════════════════════════»
                                        